<a href="https://colab.research.google.com/github/konan-91/OcularClassification/blob/master/notebooks/2_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training the Model

We now have four folders of topoplot images (which you should zip into *img_data.zip*) that will be used to train our model. We will utilise the fast.ai library - a deep learning library built ontop of PyTorch that provides high level abstractions for model training.

In [1]:
import os
from fastai.vision.all import *
from fastai.vision.widgets import *

In [2]:
!unzip -q /content/img_data.zip
!rm /content/img_data.zip

As our model's purpose is to classify blinks, we should split our data into positive and negative examples. However, we should ensure each cateogry is roughly the same size. We will use the entirety of the img_blinks dataset, and sample from img_rest, h_saccades, and v_saccades using a 50/25/25 split.

In [3]:
def count_files(path):
    count = 0
    for img in os.walk(path):
        count += 1

n_blinks = count_files('/content/img_data/img_blinks')
n_v_saccades = count_files('/content/img_data/img_n_v_saccades')
n_h_saccades = count_files('/content/img_data/img_h_saccades')
n_rest = count_files('/content/img_data/img_rest')

print(f'Number of blinks: {n_blinks}')
print(f'Number of vertical saccades: {n_v_saccades}')
print(f'Number of horizontal saccades: {n_h_saccades}')
print(f'Number of rest: {n_rest}')

Number of blinks: None
Number of vertical saccades: None
Number of horizontal saccades: None
Number of rest: None
